# Structure embedding evaluation (315 repositories / train set)

## 1. Loading repository information

In [ ]:
# Downloading repo_info_train.pkl
!pip install gdown
!gdown 13Z-ReDj4QcwlfvIgUlL6MNE5W2dcPmLr

In [ ]:
import pickle

with open("repo_info_train.pkl", "rb") as f:
    repo_info = pickle.load(f)
    f.close()

## 2. Download fine-tuning model

In [ ]:
!pip3 install torch
!pip3 install transformers
# Downloading sentence transformers
!pip3 install sentence-transformers
!wget https://raw.githubusercontent.com/microsoft/CodeBERT/master/UniXcoder/unixcoder.py
# !curl -O https://raw.githubusercontent.com/microsoft/CodeBERT/master/UniXcoder/unixcoder.py

In [ ]:
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(device)

In [ ]:
sentence_transformer_model_names = [
    "sentence-transformers/all-mpnet-base-v2",
    "sentence-transformers/multi-qa-mpnet-base-cos-v1",
    "sentence-transformers/bert-base-nli-mean-tokens",
    "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    "sentence-transformers/all-distilroberta-v1",
    "sentence-transformers/distilbert-base-nli-mean-tokens"
]

In [ ]:
from unixcoder import UniXcoder
from sentence_transformers import SentenceTransformer

sentence_transformer_models = {}

# Initialise sentence transformer model
for s_name in sentence_transformer_model_names:
    sentence_transformer_models[s_name] = SentenceTransformer(model_name_or_path=s_name, device=device)

## 3. Generating structure embeddings

In [ ]:
# Getting other embbeddings
def get_other_embeddings(data_list):
    embeddings = {}

    # Sentence transformer model
    for s_name, s_model in tqdm(sentence_transformer_models.items()):
        print(f" - Using {s_name} model - ")
        embeddings_list = s_model.encode(data_list, convert_to_tensor=True)
        embeddings[s_name] = embeddings_list

    return embeddings


# Getting embeddings
def get_embeddings(input_data=None, input_mode=None):
    if input_data is None or not input_data:
        return torch.zeros((768,), device=device)

    with torch.no_grad():
        if input_mode == "structure":
            return get_other_embeddings(input_data)


In [ ]:
from tqdm import tqdm

# Generating structure embeddings
for repo_name, repo_dict in tqdm(repo_info.items()):
    print(f" - Generating structure embeddings for {repo_name} - ")
    repo_dict["structure_embeddings"] = get_embeddings(input_data=repo_dict["structure"], input_mode="structure")

## 4. Saving structure embeddings

In [ ]:
# Change the tensor format to CPU
save_device = "cpu"

for repo_name, repo_dict in repo_info.items():
    if type(repo_dict["structure_embeddings"]) is torch.Tensor:
        repo_info[repo_name]["structure_embeddings"] = repo_info[repo_name]["structure_embeddings"].to(save_device)
    else:
        for model_name, model_tensor in repo_dict["structure_embeddings"].items():
            repo_info[repo_name]["structure_embeddings"][model_name] = model_tensor.to(save_device)

In [ ]:
# Check the tensor format
for repo_name, repo_dict in repo_info.items():
    if type(repo_dict["structure_embeddings"]) is torch.Tensor:
        print(repo_dict["structure_embeddings"].get_device())
    else:
        for model_name, model_tensor in repo_dict["structure_embeddings"].items():
             print(repo_info[repo_name]["structure_embeddings"][model_name].get_device())

In [ ]:
# Saving to repo_info_train_structure_embeddings.pkl
with open("repo_info_train_structure_embeddings.pkl", "wb") as f:
    pickle.dump(repo_info, f)
    f.close()

## 5. Calculating similarity

In [ ]:
from torch.nn import CosineSimilarity
from itertools import combinations
import pandas as pd

cossim = CosineSimilarity(dim=0, eps=1e-8)
res = []
num_of_repos = len(repo_info)
num_of_rows = num_of_repos * (num_of_repos - 1) // 2

models = sentence_transformer_model_names

for repo1, repo2 in tqdm(combinations(repo_info.keys(), 2), total=num_of_rows):
    row = [repo1, repo2, repo_info[repo1]["topic"], repo_info[repo2]["topic"]]
    for model_name in models:

        if type(repo_info[repo1]["structure_embeddings"]) is torch.Tensor:
            embedding1 = repo_info[repo1]["structure_embeddings"]
        else:
            # mean embedding
            embedding1 = torch.mean(repo_info[repo1]["structure_embeddings"][model_name], dim=0)

        if type(repo_info[repo2]["structure_embeddings"]) is torch.Tensor:
            embedding2 = repo_info[repo2]["structure_embeddings"]
        else:
            # mean embedding
            embedding2 = torch.mean(repo_info[repo2]["structure_embeddings"][model_name], dim=0)

        similarity = cossim(embedding1, embedding2).cpu().detach().numpy().item()
        row.append(similarity)

    res.append(row)

df = pd.DataFrame(res, columns=["repo1", "repo2", "topic1", "topic2"] + models)
df

In [ ]:
df.dropna(inplace=True, subset=models)
df.reset_index(drop=True, inplace=True)
df

In [ ]:
# Saving similarity calculation
df.to_csv("structure_embedding_evaluation_train_315.csv")

## 6. Evaluating different models

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

model_auc = {}
y_true = df['topic1'] == df['topic2']
fig = plt.figure(figsize=(12, 10))


def roc(model_name):
    y_score = df[model_name]
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    roc_auc = auc(fpr, tpr)
    model_auc[model_name] = roc_auc

    plt.plot(fpr, tpr, label=model_name)


for model in models:
    roc(model)

# Order labels by each model's auc score
handles, labels = plt.gca().get_legend_handles_labels()
order = sorted(range(len(labels)), key=lambda k: -model_auc[labels[k]])
ordered_handles = [handles[idx] for idx in order]
ordered_labels = [f'{labels[idx]} (AUC = {model_auc[labels[idx]]:.3f})' for idx in order]
plt.legend(ordered_handles, ordered_labels, loc="lower right")

plt.plot([0, 1], [0, 1], 'k--', label='Random guess')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'Receiver Operating Characteristic of different models')
# Saving evaluation result
plt.savefig('roc_structure_evaluation_train_315.png')
plt.show()

In [ ]:
model_auc